In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error

In [2]:
regr=linear_model.LinearRegression()


In [3]:
data=pd.read_csv('houses_train.csv')
target=data['price']
data.head()
print(list(data.columns))
data['region'].unique()




['Unnamed: 0', 'price', 'condition', 'district', 'max_floor', 'street', 'num_rooms', 'region', 'area', 'url', 'num_bathrooms', 'building_type', 'floor', 'ceiling_height']


array(['Yerevan'], dtype=object)

In [5]:
data.head()

,Unnamed: 0,price,condition,district,max_floor,street,num_rooms,region,area,url,num_bathrooms,building_type,floor,ceiling_height
0,5546,130000.0,newly repaired,Center,4,Sayat Nova Ave,3,Yerevan,96.0,http://www.myrealty.am/en/item/28244/3-senyaka...,1,stone,3,3.2
1,2979,65000.0,good,Arabkir,5,Hr.Kochar St,3,Yerevan,78.0,http://www.myrealty.am/en/item/18029/3-senyaka...,1,stone,2,2.8
2,2698,129000.0,good,Center,10,M.Khorenatsi St,3,Yerevan,90.0,http://www.myrealty.am/en/item/37797/3-senyaka...,1,panel,3,2.8
3,4548,52000.0,newly repaired,Center,14,Argishti St,2,Yerevan,53.0,http://www.myrealty.am/en/item/36153/2-senyaka...,1,monolit,5,3.0
4,2982,65000.0,newly repaired,Center,12,Mashtots Ave,2,Yerevan,47.0,http://www.myrealty.am/en/item/17566/2-senyaka...,1,panel,3,2.8


## Using linear regression

In [7]:
df1=data[["Unnamed: 0",'max_floor','num_rooms','area','num_bathrooms','floor','ceiling_height']]
df1

,Unnamed: 0,max_floor,num_rooms,area,num_bathrooms,floor,ceiling_height
0,5546,4,3,96.0,1,3,3.2
1,2979,5,3,78.0,1,2,2.8
2,2698,10,3,90.0,1,3,2.8
3,4548,14,2,53.0,1,5,3.0
4,2982,12,2,47.0,1,3,2.8
...,...,...,...,...,...,...,...
4995,2417,16,3,100.0,1,8,2.8
4996,1307,6,3,135.0,1,5,2.8
4997,2863,9,3,73.0,2,3,3.0
4998,1470,5,1,48.0,1,1,2.8


In [8]:
xTrain,xTest,yTrain,yTest=train_test_split(df1,target,random_state=1,test_size=0.2
                                          )

In [36]:
regr.fit(xTrain,yTrain)
pred1=regr.predict(xTest)
pred11=regr.predict(xTrain)
pred1[:5]

array([120918.78274842,  68449.50549587,  28573.21118923,  70875.90287766,
        56296.16761546])

In [37]:
yTest[:5]

2764     62000.0
4767     37000.0
3814     55000.0
3499    118000.0
2735     42000.0
Name: price, dtype: float64

In [38]:
regr.coef_, regr.intercept_

(array([-3.71271120e-02, -1.02285813e+02, -1.24985145e+04,  1.35693020e+03,
         1.90831916e+04,  1.01326874e+02,  7.89021625e+04]),
 -241765.10707379336)

In [39]:
print ('Test RMSE:', np.sqrt(mean_squared_error(pred1, yTest)))
print ('Train RMSE:', np.sqrt(mean_squared_error(pred11, yTrain)))

Test RMSE: 36168.1455396416
Train RMSE: 36833.17948704149


## Using KFold validation

In [40]:
df1.head()

,Unnamed: 0,max_floor,num_rooms,area,num_bathrooms,floor,ceiling_height
0,5546,4,3,96.0,1,3,3.2
1,2979,5,3,78.0,1,2,2.8
2,2698,10,3,90.0,1,3,2.8
3,4548,14,2,53.0,1,5,3.0
4,2982,12,2,47.0,1,3,2.8


In [60]:

from sklearn.model_selection import KFold
kf = KFold(n_splits=10) # Define the split - into 10 folds 
test_list=[]
train_list=[]
for train_index, test_index in kf.split(df1):
    #print('TRAIN:', train_index, 'TEST:', test_index)
    X_train, X_test = df1.iloc[train_index], df1.iloc[test_index]
    y_train, y_test = target[train_index], target[test_index]
    regr.fit(X_train, y_train)
    predictions1 = regr.predict(X_test)
    predictions2 = regr.predict(X_train)
    test_list.append(np.sqrt(mean_squared_error(predictions1,y_test)))
    train_list.append(np.sqrt(mean_squared_error(predictions2,y_train)))
print("mean of test" ,np.mean(test_list))
print("mean of train" ,np.mean(train_list))

mean of test 36733.51604261687
mean of train 36686.95912261126


## Using Standard Scaler

In [42]:
from sklearn.preprocessing import StandardScaler

In [43]:
norm=StandardScaler().fit(xTrain)
dfTrain=pd.DataFrame(columns=xTrain.columns, data=norm.transform(xTrain))
dfTrain.head()

,Unnamed: 0,max_floor,num_rooms,area,num_bathrooms,floor,ceiling_height
0,0.979334,-0.880765,1.577249,0.609944,1.998530,-0.034051,2.060643
1,-1.086931,-0.880765,0.364214,1.536146,-0.414425,-0.034051,-0.657883
2,-0.115066,2.477272,1.577249,0.771023,4.411485,3.230090,0.701380
3,-1.491874,1.757692,0.364214,0.730753,-0.414425,1.152909,0.701380
4,0.949358,1.757692,1.577249,0.771023,-0.414425,3.230090,0.701380


In [44]:
dfTest=pd.DataFrame(columns=xTest.columns, data=norm.transform(xTest))
dfTest.head()

,Unnamed: 0,max_floor,num_rooms,area,num_bathrooms,floor,ceiling_height
0,0.279886,1.277973,-0.848821,0.086439,1.998530,-0.330791,0.701380
1,-1.417722,1.757692,0.364214,-0.074640,-0.414425,0.856169,-0.657883
2,-0.437443,0.078674,-0.848821,-1.645156,-0.414425,0.559429,-0.657883
3,-0.868156,-1.120625,-0.848821,-0.396797,-0.414425,-0.330791,-0.657883
4,-1.016986,0.798253,-0.848821,-0.799493,-0.414425,-0.924271,-0.657883


In [45]:
regr.fit(dfTrain,yTrain)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [46]:
print("Test RMSE after Standard Scaler", np.sqrt(mean_squared_error(regr.predict(dfTest), yTest)))
print("Train RMSE after Standard Scaler", np.sqrt(mean_squared_error(regr.predict(dfTrain), yTrain)))

Test RMSE after Standard Scaler 36168.14553964143
Train RMSE after Standard Scaler 36833.17948704149


## Using One Hot Encoding

In [47]:
data2= data.drop('region',axis=1)

In [48]:
data2.head()

,Unnamed: 0,price,condition,district,max_floor,street,num_rooms,area,url,num_bathrooms,building_type,floor,ceiling_height
0,5546,130000.0,newly repaired,Center,4,Sayat Nova Ave,3,96.0,http://www.myrealty.am/en/item/28244/3-senyaka...,1,stone,3,3.2
1,2979,65000.0,good,Arabkir,5,Hr.Kochar St,3,78.0,http://www.myrealty.am/en/item/18029/3-senyaka...,1,stone,2,2.8
2,2698,129000.0,good,Center,10,M.Khorenatsi St,3,90.0,http://www.myrealty.am/en/item/37797/3-senyaka...,1,panel,3,2.8
3,4548,52000.0,newly repaired,Center,14,Argishti St,2,53.0,http://www.myrealty.am/en/item/36153/2-senyaka...,1,monolit,5,3.0
4,2982,65000.0,newly repaired,Center,12,Mashtots Ave,2,47.0,http://www.myrealty.am/en/item/17566/2-senyaka...,1,panel,3,2.8


In [49]:
data2.columns

Index(['Unnamed: 0', 'price', 'condition', 'district', 'max_floor', 'street',
       'num_rooms', 'area', 'url', 'num_bathrooms', 'building_type', 'floor',
       'ceiling_height'],
      dtype='object')

In [50]:
len(data2.columns)

13

In [51]:
data_one_hot_encoding=pd.get_dummies(data2[['Unnamed: 0','condition','district','max_floor','street','num_rooms', 'area', 'url', 'num_bathrooms', 'building_type', 'floor',
       'ceiling_height']])


In [52]:
data_one_hot_encoding.head()


,Unnamed: 0,max_floor,num_rooms,area,num_bathrooms,floor,ceiling_height,condition_good,condition_newly repaired,condition_zero condition,...,url_http://www.myrealty.am/en/item/98/2-senyakanoc-bnakaran-vacharq-Yerevan-Arabkir,url_http://www.myrealty.am/en/item/99/2-senyakanoc-bnakaran-vacharq-Yerevan-Center,url_http://www.myrealty.am/en/item/9923/2-senyakanoc-bnakaran-vacharq-Yerevan-Center,url_http://www.myrealty.am/en/item/9934/apartment-for-sale-in-Yerevan,url_http://www.myrealty.am/en/item/9937/2-senyakanoc-bnakaran-vacharq-Yerevan-Arabkir,url_http://www.myrealty.am/en/item/9953/apartment-for-sale-in-Yerevan,building_type_monolit,building_type_other,building_type_panel,building_type_stone
0,5546,4,3,96.0,1,3,3.2,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,2979,5,3,78.0,1,2,2.8,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2698,10,3,90.0,1,3,2.8,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,4548,14,2,53.0,1,5,3.0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,2982,12,2,47.0,1,3,2.8,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [53]:
trainX,testX,trainY,testY=train_test_split(data_one_hot_encoding,target,test_size=0.2,random_state=1)
print(len(trainX),len(trainY),len(testX),len(testY))

4000 4000 1000 1000


In [57]:
regr.fit(trainX,trainY)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [58]:
print('Test RSME after one hot encoding',np.sqrt(mean_squared_error(regr.predict(testX),testY)))
print('Train RMSE after one hot encoding', np.sqrt(mean_squared_error(regr.predict(trainX), trainY)))

Test RSME after one hot encoding 22486.00434540876
Train RMSE after one hot encoding 8.342734298942872e-08


In [59]:
regr.coef_, regr.intercept_

(array([-2.49181351e-01,  4.23797230e+02, -2.47941360e+03, ...,
         2.00678901e+03, -9.99829435e+03, -3.35137959e+03]),
 -75322.2353523129)

## Using Ridge Regularization

In [61]:
from sklearn.linear_model import Ridge

In [68]:
rr=Ridge(alpha=10,random_state=1)
rr.fit(xTrain,yTrain)

Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=1, solver='auto', tol=0.001)

In [69]:
print("Test RMSE after ridge reg",np.sqrt(mean_squared_error(rr.predict(xTest),yTest)))
print("Train RMSE after ridge reg",np.sqrt(mean_squared_error(rr.predict(xTrain),yTrain)))

Test RMSE after ridge reg 36162.85093519074
Train RMSE after ridge reg 36853.85876539976


In [75]:
rr=Ridge(alpha=50,random_state=1)
rr.fit(xTrain,yTrain)

Ridge(alpha=50, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=1, solver='auto', tol=0.001)

In [76]:
print("Test RMSE after ridge reg",np.sqrt(mean_squared_error(rr.predict(xTest),yTest)))
print("Train RMSE after ridge reg",np.sqrt(mean_squared_error(rr.predict(xTrain),yTrain)))

Test RMSE after ridge reg 36324.49027955345
Train RMSE after ridge reg 37080.597837017114


In [86]:
rr=Ridge(alpha=100,random_state=1)
rr.fit(xTrain,yTrain)
rr.coef_

array([ 9.17775603e-02, -2.49924362e+02, -1.29203881e+04,  1.43265753e+03,
        1.83790953e+04,  1.04645187e+02,  3.54543431e+04])

In [87]:
print("Test RMSE after ridge reg",np.sqrt(mean_squared_error(rr.predict(xTest),yTest)))
print("Train RMSE after ridge reg",np.sqrt(mean_squared_error(rr.predict(xTrain),yTrain)))


Test RMSE after ridge reg 36569.204833800504
Train RMSE after ridge reg 37354.74617982128


In [84]:
## Using Ridge reg with one hot encoding

In [90]:
rr=Ridge(alpha=100,random_state=1)
rr.fit(trainX,trainY)
rr.coef_

array([-1.27551842e-01,  1.41271821e+02, -5.31443195e+03, ...,
        1.20342825e+03, -1.15773963e+04, -1.10480104e+03])

In [82]:
print("Test RMSE after ridge reg",np.sqrt(mean_squared_error(rr.predict(testX),testY)))
print("Train RMSE after ridge reg",np.sqrt(mean_squared_error(rr.predict(trainX),trainY)))

Test RMSE after ridge reg 26908.400877550055
Train RMSE after ridge reg 26970.010907556254


## Using Lasso regularization

In [91]:
from sklearn.linear_model import Lasso

In [101]:
lr=Lasso(alpha=10,random_state=1)

In [102]:

lr.fit(xTrain,yTrain)

Lasso(alpha=10, copy_X=True, fit_intercept=True, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=1, selection='cyclic',
      tol=0.0001, warm_start=False)

In [103]:
print("Test RMSE after lasso reg",np.sqrt(mean_squared_error(lr.predict(xTest),yTest)))
print("Train RMSE after lasso reg",np.sqrt(mean_squared_error(lr.predict(xTrain),yTrain)))

Test RMSE after lasso reg 36168.93362464281
Train RMSE after lasso reg 36833.250806292934


In [110]:
lr=Lasso(alpha=100,random_state=1)
lr.fit(xTrain,yTrain)

Lasso(alpha=100, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=1,
      selection='cyclic', tol=0.0001, warm_start=False)

In [112]:
print("Test RMSE after lasso reg",np.sqrt(mean_squared_error(lr.predict(xTest),yTest)))
print("Train RMSE after lasso reg",np.sqrt(mean_squared_error(lr.predict(xTrain),yTrain)))

Test RMSE after lasso reg 36181.79280139541
Train RMSE after lasso reg 36840.30922255961


In [ ]:
## ## Using Lasso reg with one hot encoding

In [ ]:
lr=Lasso(alpha=100,random_state=1)
lr.fit(trainX,trainY)

In [113]:
print("Test RMSE after lasso reg",np.sqrt(mean_squared_error(rr.predict(testX),testY)))
print("Train RMSE after lasso reg",np.sqrt(mean_squared_error(rr.predict(trainX),trainY)))

Test RMSE after lasso reg 26908.400877550055
Train RMSE after lasso reg 26970.010907556254


### After using "one hot encoding" Lasso and Ridge reg results are the same

Test RMSE after lasso reg 26908.400877550055
Train RMSE after lasso reg 26970.010907556254

Test RMSE after ridge reg 26908.400877550055
Train RMSE after ridge reg 26970.010907556254